In [ ]:
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
import statsmodels.api as sm
import patsy
import gc 

df = pd.read_csv('/unequal_outage_data_redacted.csv') 

df_filtered = df[['anonymized_zip_id', 'date','anonymized_county', 'ln_customers_duration', 'heavy_ppt1','heat', 'cold', 'heavy_snow_fall', 'cyclone_binary', 'strong_wind', 'fire_binary',
        'total_customers_served', 'state','substation_count', 'distance_to_nearest_substation', 'population_density', 'coastal', 'urban',
         'tree_coverage_percentage', 'per_built_after_2010', 'per_built_1990_2009', 'per_built_1970_1989', 'per_built_1950_1969', 'ln_median_hhi', 'per_nonwhite',
         'water_treatment', 'hospital', 'nursing_home' ,'per_old' ,'per_electricity'
        ]]

# Define target 
target_column = 'ln_customers_duration'

# 2. FEATURE ENGINEERING
# Define the weather and other variables to analyze
weather_vars = ['cyclone_binary', 'strong_wind', 'fire_binary', 'heavy_ppt1', 
                'heat', 'cold', 'heavy_snow_fall']

# Combine all predictors
predictors = weather_vars

# 3. MISSING VALUES
df_clean = df_filtered.copy()

# Drop rows with missing values in our key variables
missing_before = len(df_clean)
df_clean = df_clean.dropna(subset=[target_column] + predictors)
missing_after = len(df_clean)

# Create anonymized_zip_id and date dummies for fixed effects
df_clean['anonymized_zip_id'] = df_clean['anonymized_zip_id'].astype(str)
df_clean['date'] = pd.to_datetime(df_clean['date']).dt.date.astype(str)

# 5. IMPLEMENTING PANEL REGRESSION WITH FIXED EFFECTS
# Build formula string for regression with fixed effects
formula_parts = [target_column, '~'] + predictors + ['C(anonymized_zip_id)', 'C(date)']
formula = ' + '.join(formula_parts)


try:
    print("Preparing data matrix for fixed effects model...")
    y = df_clean[target_column]
    
    # Remove sparse parameter as it's not supported in your patsy version
    X = patsy.dmatrix(formula.split('~')[1], data=df_clean, return_type='dataframe')
    
    # Add constant
    X = sm.add_constant(X)
    
    # Run regression with fixed effects
    print("Running fixed effects regression...")
    model = sm.OLS(y, X).fit(cov_type='cluster', cov_kwds={'groups': df_clean['anonymized_zip_id']})
    
    # Print results
    print("\nFixed Effects Regression Results:")
    print(model.summary())
    
    # Save the regression results to a text file
    with open('fixed_effects_results.txt', 'w') as f:
        f.write(model.summary().as_text())
    
    print("1. fixed_effects_results.txt - Fixed effects regression results")
    
    # Free up memory
    del X
    gc.collect()
except Exception as e:
    print(f"Error in fixed effects model: {str(e)}")
    print("Skipping main fixed effects model and proceeding with anonymized_zip_id-specific analysis...")

# 6. anonymized_zip_id-SPECIFIC ANALYSIS WITHOUT INTERACTION MODEL
print("\nAnalyzing anonymized_zip_id-specific weather effects (no interaction model)...")

# Reduce memory usage by using only necessary columns
essential_columns = [target_column, 'anonymized_zip_id', 'date'] + predictors
df_minimal = df_clean[essential_columns].copy()

# Alternative approach: Run separate regressions for each anonymized_zip_id
all_anonymized_zip_ids = df_minimal['anonymized_zip_id'].unique().tolist()
print(f"Total anonymized_zip_ids to analyze: {len(all_anonymized_zip_ids)}")

# Prepare a dataframe to store coefficients for each anonymized_zip_id
anonymized_zip_id_coefficients = pd.DataFrame(columns=['anonymized_zip_id'] + weather_vars)

# Process anonymized_zip_ids in batches to manage memory
BATCH_SIZE = 50
num_batches = (len(all_anonymized_zip_ids) + BATCH_SIZE - 1) // BATCH_SIZE

for batch_idx in range(num_batches):
    batch_start = batch_idx * BATCH_SIZE
    batch_end = min((batch_idx + 1) * BATCH_SIZE, len(all_anonymized_zip_ids))
    batch_anonymized_zip_ids = all_anonymized_zip_ids[batch_start:batch_end]
    
    print(f"\nProcessing batch {batch_idx+1}/{num_batches} with {len(batch_anonymized_zip_ids)} anonymized_zip_ids...")
    
    # Process each anonymized_zip_id in the batch
    for anonymized_zip_id in tqdm(batch_anonymized_zip_ids):
        # Filter data for this anonymized_zip_id
        anonymized_zip_id_data = df_minimal[df_minimal['anonymized_zip_id'] == anonymized_zip_id].copy()
        
        # Skip if too few observations
        if len(anonymized_zip_id_data) < 30:  # Minimum observations for meaningful regression
            print(f"Skipping anonymized_zip_id {anonymized_zip_id}: insufficient observations ({len(anonymized_zip_id_data)})")
            continue
            
        try:
            # Create regression formula for this anonymized_zip_id
            # We don't need anonymized_zip_id fixed effects when analyzing a single anonymized_zip_id
            anonymized_zip_id_formula_parts = [target_column, '~'] + predictors + ['C(date)']
            anonymized_zip_id_formula = ' + '.join(anonymized_zip_id_formula_parts)
            
            # Prepare data
            y_zip = anonymized_zip_id_data[target_column]
            X_zip = patsy.dmatrix(anonymized_zip_id_formula.split('~')[1], data=anonymized_zip_id_data, return_type='dataframe')
            X_zip = sm.add_constant(X_zip)
            
            # Run regression for this anonymized_zip_id
            anonymized_zip_id_model = sm.OLS(y_zip, X_zip).fit()
            
            # Extract weather coefficients
            coef_row = {'anonymized_zip_id': anonymized_zip_id}
            for var in weather_vars:
                # Get coefficient if it exists in the model
                try:
                    coef_row[var] = anonymized_zip_id_model.params[var]
                except:
                    # The variable might have a prefix in the model
                    try:
                        param_name = [name for name in anonymized_zip_id_model.params.index if name.endswith(var)]
                        if param_name:
                            coef_row[var] = anonymized_zip_id_model.params[param_name[0]]
                        else:
                            coef_row[var] = np.nan
                    except:
                        coef_row[var] = np.nan
                    
            # Add to results dataframe
            anonymized_zip_id_coefficients = pd.concat([anonymized_zip_id_coefficients, 
                                             pd.DataFrame([coef_row])], 
                                             ignore_index=True)
                
        except Exception as e:
            print(f"Error processing anonymized_zip_id {anonymized_zip_id}: {str(e)}")
            continue
            
        # Clean up to save memory
        del anonymized_zip_id_data, X_zip, anonymized_zip_id_model
        gc.collect()
    
    print(f"Completed batch {batch_idx+1}/{num_batches}")

# 7. PROCESS AND VISUALIZE THE RESULTS
print("\nProcessing results and creating visualizations...")

if len(anonymized_zip_id_coefficients) > 0:
    # Save the full coefficients data to CSV
    print("Saving anonymized_zip_id coefficients to CSV...")
    anonymized_zip_id_coefficients.to_csv('anonymized_zip_id_weather_coefficients.csv', index=False)
    
    # Set anonymized_zip_id as index for heatmap
    heatmap_data = anonymized_zip_id_coefficients.set_index('anonymized_zip_id')
    
    # Create a cleaner version for analysis
    vulnerability_scores = heatmap_data.copy()
    # For vulnerability, we're interested in negative coefficients (more disruption)
    vulnerability_scores['avg_vulnerability'] = -vulnerability_scores[weather_vars].mean(axis=1)
    vulnerability_scores = vulnerability_scores.sort_values('avg_vulnerability', ascending=False)
    
    # Save this sorted version
    vulnerability_scores.to_csv('anonymized_zip_id_vulnerability_scores.csv')
    
    # For the heatmap, limit to top 30 anonymized_zip_ids by vulnerability
    top_vulnerable_anonymized_zip_ids = vulnerability_scores.head(30).index.tolist()
    if len(top_vulnerable_anonymized_zip_ids) > 0:
        top_heatmap_data = heatmap_data.loc[top_vulnerable_anonymized_zip_ids, weather_vars]
        
        # Plot heatmap for top 30 most vulnerable anonymized_zip_ids
        plt.figure(figsize=(16, 12))
        sns.heatmap(top_heatmap_data, cmap='RdBu_r', center=0, annot=True, fmt='.3f')
        plt.title('Top 30 Most Vulnerable anonymized_zip_ids to Weather Events')
        plt.xlabel('Weather Variables')
        plt.ylabel('anonymized_zip_ids')
        plt.tight_layout()
        plt.savefig('top30_anonymized_zip_id_weather_vulnerability.png')
    
    # Create a full heatmap only if it's a reasonable size
    if len(anonymized_zip_id_coefficients) < 200:  # Limit full heatmap to manageable size
        plt.figure(figsize=(20, 24))
        heatmap_all = heatmap_data[weather_vars]
        sns.heatmap(heatmap_all, cmap='RdBu_r', center=0, annot=False)
        plt.title('All anonymized_zip_id Vulnerability to Weather Events')
        plt.xlabel('Weather Variables')
        plt.ylabel('anonymized_zip_ids')
        plt.tight_layout()
        plt.savefig('all_anonymized_zip_id_weather_vulnerability.png')
    else:
        print(f"Skipping full heatmap generation due to large number of anonymized_zip_ids ({len(anonymized_zip_id_coefficients)})")
    
    # Create summary statistics for each weather variable
    print("\nGenerating summary statistics on anonymized_zip_id vulnerability by weather type...")
    weather_vulnerability_summary = pd.DataFrame({
        'weather_type': weather_vars,
        'mean_effect': [heatmap_data[var].mean() for var in weather_vars],
        'median_effect': [heatmap_data[var].median() for var in weather_vars],
        'std_dev': [heatmap_data[var].std() for var in weather_vars],
        'min_effect': [heatmap_data[var].min() for var in weather_vars],
        'max_effect': [heatmap_data[var].max() for var in weather_vars],
        'most_vulnerable_anonymized_zip_id': [heatmap_data[var].idxmin() for var in weather_vars],
        'least_vulnerable_anonymized_zip_id': [heatmap_data[var].idxmax() for var in weather_vars]
    })
    
    # Save the summary statistics
    weather_vulnerability_summary.to_csv('weather_vulnerability_summary.csv', index=False)
    
    if len(anonymized_zip_id_coefficients) < 200:
        print("5. all_anonymized_zip_id_weather_vulnerability.png - Full heatmap of all anonymized_zip_id vulnerabilities")
    
    if 'model' in locals():
        print("6. fixed_effects_results.txt - Main fixed effects regression results")
else:
    print("Warning: No anonymized_zip_id coefficients were successfully extracted. Check for errors in the batch processing.")

Preparing data matrix for fixed effects model...
